<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Copy_of_Antim_Bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# tf.config.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[0], 'GPU')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import tensorflow as tf

In [7]:
import pandas as pd

In [8]:
df=pd.read_csv('/content/drive/MyDrive/data.csv')

In [9]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


In [10]:
df.dropna(inplace=True)

In [11]:
df1=df[:1000]

In [12]:
df1.to_csv('data1.csv',index=False)

In [13]:
data=pd.read_csv('data1.csv')

In [14]:
data.shape

(1000, 6)

**Split the dataset into train and evaluate**

In [15]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2)

In [16]:
train_data.shape

(800, 6)

In [17]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')


In [18]:
import csv

# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [19]:
# import csv

# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [20]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.7 MB/s eta 0:00:00


In [21]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [22]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [23]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)



/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [24]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


In [25]:
# Set up the trainer with the appropriate training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    save_total_limit=2,
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset
)



In [26]:
# Fine-tune the model on your dataset
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.443925
2,3.535900,3.411552
3,3.535900,3.410020


TrainOutput(global_step=927, training_loss=3.424958888509641, metrics={'train_runtime': 555.6499, 'train_samples_per_second': 26.617, 'train_steps_per_second': 1.668, 'total_flos': 966127288320000.0, 'train_loss': 3.424958888509641, 'epoch': 3.0})

In [27]:
trainer.evaluate()

{'eval_loss': 3.410019874572754,
 'eval_runtime': 13.8455,
 'eval_samples_per_second': 93.605,
 'eval_steps_per_second': 11.701,
 'epoch': 3.0}

In [28]:
trainer

In [29]:
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [30]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt').to('cuda'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


which is the best phone that i can buy?  pic.twitter.com/ZdGk7J4D1S —  Goyal (@Goyal)   August 18, 2020 More of this stuff in an earlier post. It could help explain why Samsung’s flagships have been in a lockdown for some time. Image Credits:  Samsung Electronics A reminder that the device was the first model for the forthcoming 4G/GPRS wireless infrastructure — which is set to roll out in Q1 2020. Samsung’s Galaxy Z Ultra is another example of a device that can now include a fingerprint sensor along with a fingerprint reader, but we’ve taken a bit more of a more nuanced look at the device’s internals, and we’ll go back to what we’ve seen below: The fingerprint sensor, meanwhile, isn’t that far behind the Galaxy Z Series 2 and Z Ultra Ultra 3. Image Credits:


In [31]:
trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [33]:
import joblib

joblib.dump(model, 'SecondModel_Epoch_3_1000.joblib')

['SecondModel_Epoch_3_1000.joblib']

In [34]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000.joblib')

In [36]:
loaded_model_joblib

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [37]:
import joblib as jb

In [39]:
jb.dump(model,'model_jb.pkl')

['model_jb.pkl']

In [40]:
loaded_model = jb.load('model_jb.pkl')

In [41]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = loaded_model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt').to('cuda'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


which is the best phone that i can buy? and what can you say?) has launched its own program in China. You can find it on  Google Play,  Apple’s App Store and  the  Chinese web. In its most recent call, the two companies called for an end to the illegal spread of Xiaomi Mi Mix. The company also pledged to bring the Xiaomi Mi Mix 2  to market and would provide support and customer support if necessary. “I think that this is definitely a big step forward for us,” he said. Xiaomi is one of the world’s largest smartphone companies. Its flagship smartphone, the Mi Flex 2, was first unveiled in the United States and North America last fall, and in 2018 it hit the market in an all-handson-one production run in India, India in a single order, and China in multiple orders. Xiaomi debuted the Mi Flex 2 at the Mobile World Congress this past March. There are now more than
